In [4]:
# Primero, importamos las librerías que necesitaremos
import requests  # La librería estándar en Python para hacer peticiones a la web
import pandas as pd
import numpy as np

# 1. Definimos los parámetros de nuestra consulta
codigos = 'PN38705PM-PD04722MM-PN07819NM-PN06522TM-PN06534TM-PN06502OM-PN06503OM-PN07150NM-PD04724MM' # Códigos de las series, separados por guion: Inflación mensual (variación % IPC), Tasa de interés de referencia (TIR), Tasa de interés interbancaria promedio, *repo de valores saldo, depósitos públicos saldo,CD BCRP - Total - Saldo
# las otras series: CD BCRP - Tasa de ínteres - Saldo, Depositos overnight BCRP (promedio diario) - Empresas Bancarias (miles S/), Tasa de los Depósitos Overnight en MN
formato = 'json' # Pedimos los datos en formato JSON
periodo_inicial = '2010-01'
periodo_final = '2025-08' # Ponemos un periodo final lejano para obtener siempre los últimos datos

# 2. Construimos la URL completa
#Como habiamos visto en la clase pasada, usamos f"" para crear una concatenación de strings (textos) uniendo otros objetos.
url = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigos}/{formato}/{periodo_inicial}/{periodo_final}"

# 3. Hacemos la petición GET a la web del BCRP
response = requests.get(url)

# 4. Verificamos que la petición fue exitosa (un código 200 significa que todo salió bien)
if response.status_code == 200:
    print("¡Conexión exitosa con el API del BCRP!")
    # Convertimos la respuesta a un formato de diccionario de Python
    data = response.json()
else:
    print(f"Error al conectar con el API. Código de estado: {response.status_code}")
    data = None

¡Conexión exitosa con el API del BCRP!


In [5]:
# Miremos los datos de los primeros dos periodos para entender la estructura
# [tasa de interés de ref, tasa de depósitos overnight, CDBCR total saldo, CDBCR saldo tasa de interés, repos de valores, depósitos públicos, depósitos overnight BCR,  tasa de interés interbancaria, IPC]
if data:
    print(data['periods'][0])
    print(data['periods'][1])
    print(data['periods'][-1])

{'name': 'Ene.2010', 'values': ['1.25', '0.45', '19352.2', '1.48499168053245', '0', '0', 'n.d.', '1', '69.7136816328321']}
{'name': 'Feb.2010', 'values': ['1.25', '0.45', '20057.9', '1.48248012005245', '0', '0', 'n.d.', '1.21524', '69.9385138944542']}
{'name': 'Ago.2025', 'values': ['4.5', '2.5', '43991.9', '4.2065169497112', '7306.947803', '4024.9', '78929.5483870968', '4.5079', '115.586828']}


In [8]:
if data:
    # Creamos listas vacías
    fechas = []
    tir = []
    tover = []
    cdbcr_total = []
    cdbcr_tir = []
    repos_valores = []
    depositos_publicos = []
    depositos_overnight = []
    interbancaria = []
    ipc = []


    # Iteramos sobre los periodos en los datos del BCRP
    for periodo in data['periods']:
        fechas.append(periodo['name'])

        # Los valores vienen en el mismo orden en que pusiste los códigos en la URL:
        # PN38705PM - PD04722MM - PN07839NM - PN06525TM - PN06534TM
        valores = periodo['values']

        # Tasa de interés de referencia
        tir.append(float(valores[0]) if valores[0] != 'n.d.' else np.nan)
        # Depositos overnight
        tover.append(float(valores[1]) if valores[1] != 'n.d.' else np.nan)
        # CDBCR total saldo
        cdbcr_total.append(float(valores[2]) if valores[2] != 'n.d.' else np.nan)
        # CDBCR saldo tasa de interés
        cdbcr_tir.append(float(valores[3]) if valores[3] != 'n.d.' else np.nan)
       # Repos (saldo)
        repos_valores.append(float(valores[4]) if valores[4] != 'n.d.' else np.nan)
       # Depósitos públicos (saldo)
        depositos_publicos.append(float(valores[5]) if valores[5] != 'n.d.' else np.nan)
        # Depositos overnight BCR
        depositos_overnight.append(float(valores[6]) if valores[6] != 'n.d.' else np.nan)
        # Tasa interbancaria promedio
        interbancaria.append(float(valores[7]) if valores[7] != 'n.d.' else np.nan)
        # IPC (variación %)
        ipc.append(float(valores[8]) if valores[8] != 'n.d.' else np.nan)

            # Construimos el DataFrame
    datos_para_df = {
        'Fecha': fechas,
        'Tasa_Ref (%)': tir,
        'Tasa_Depositos_Overnight (%)': tover,
        'CDBCR_Total_Saldo (S/ millones)': cdbcr_total,
        'CDBCR_Saldo_TIR (%)': cdbcr_tir,
        'Repos_Valores (S/ millones)': repos_valores,
        'Depositos_Publicos_Saldo (S/ millones)': depositos_publicos,
        'Depositos_Overnight_BCR (S/ miles)': depositos_overnight,
        'Tasa_Interbancaria (%)': interbancaria,
        'IPC (%)': ipc,

    }

    df_bcrp = pd.DataFrame(datos_para_df)

    print("DataFrame creado exitosamente:")
    display(df_bcrp.head())
else:
    print("No se pudieron cargar los datos para crear el DataFrame.")


DataFrame creado exitosamente:


,Fecha,Tasa_Ref (%),Tasa_Depositos_Overnight (%),CDBCR_Total_Saldo (S/ millones),CDBCR_Saldo_TIR (%),Repos_Valores (S/ millones),Depositos_Publicos_Saldo (S/ millones),Depositos_Overnight_BCR (S/ miles),Tasa_Interbancaria (%),IPC (%)
0,Ene.2010,1.25,0.45,19352.2,1.484992,0.0,0.0,NaN,1.00000,69.713682
1,Feb.2010,1.25,0.45,20057.9,1.482480,0.0,0.0,NaN,1.21524,69.938514
2,Mar.2010,1.25,0.45,21752.6,1.496076,0.0,0.0,NaN,1.23264,70.134910
3,Abr.2010,1.25,0.45,19070.7,1.438794,0.0,0.0,NaN,1.21970,70.152694
4,May.2010,1.50,0.70,17733.4,1.356306,0.0,0.0,NaN,1.44610,70.319559
